In this tutorial, we're going to attempt to estimate the mass of the planets in the Kepler-29 system from TTVs, comparing to the results of Jontof-Hutter et al. (2016). From Table 4, Test 3 of that paper (which also uses the Holczer catalog), we're looking for a dynamical masses of 3.3(+1.8)(-1.7) and 2.6(+1.4)(-1.3), respectively (where the error bars represent 1sigma credible intervals).

First, we'll set everything up and import ttvnest:

In [ ]:
%matplotlib inline
import numpy as np
import ttvnest
import scipy

Now, let's download the data for Kepler-29 (KOI-738) from the Holczer et al. (2016) catalog:

In [ ]:
koi = 738
nplanets = 2
catfile = 'ttvnest/holczer2016.txt'
data, errs = ttvnest.load_holczer.get_data(catfile, koi, nplanets)

Next, we need to define the prior transform. Nested sampling methods typically sample on the unit cube (from 0 to 1 in all dimensions), so we need a way to map that into our preferred prior space. Eventually, I'll try to abstract the prior transform away from the user.

In [ ]:
#defining the prior transform

def prior_transform(u):
	x = np.array(u) #copy u
	#planet 1
	x[0] = 10.*u[0] #uniform on [0, 10) earth masses
	x[1] = scipy.stats.norm.ppf(u[1], 10.33585, 0.01)  #gaussian around 10.33585 with a width of 0.01
	x[2] = scipy.stats.norm.ppf(u[2], 0, 0.1) #gaussian around 0 with a width of 0.1
	x[3] = scipy.stats.norm.ppf(u[3], 0, 0.1) #gaussian around 0 with a width of 0.1
	x[4] = (u[4] % 1.) * 360. #periodic on [0, 360) degrees
	#planet 2
	x[5] = 10.*u[5] #uniform on [0, 10) earth masses
	x[6] = scipy.stats.norm.ppf(u[6], 13.29292, 0.01) #gaussian around 13.29292 with a width of 0.01
	x[7] = scipy.stats.norm.ppf(u[7], 0, 0.1) #gaussian around 0 with a width of 0.1
	x[8] = scipy.stats.norm.ppf(u[8], 0, 0.1) #gaussian around 0 with a width of 0.1
	x[9] = (u[9] % 1.) * 360. #periodic on [0, 360) degrees
	return x

Finally, we sample the posteriors!

In [ ]:
%time results = ttvnest.retrieval.retrieve(nplanets, prior_transform, data, errs)

Let's now see a summary of our results:

In [ ]:
ttvnest.retrieval.posterior_summary(results)

And let's plot the result, as well as the corner plot, a summary of the run, and the dynesty trace plot.

In [ ]:
ttvnest.plot_utils.plot_results(results, data, errs)
ttvnest.plot_utils.dynesty_plots(results, nplanets)

Let's make sure to save our results! The results will be saved in a file called 'results.p' by default, but you can change that with the "outname" keyword for save_results.

In [ ]:
ttvnest.io_utils.save_results(results)